In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import numpy as np
import pandas as pd
import hashlib
import itertools
import random
import sys
from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix, csc_matrix, coo_matrix, lil_matrix

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [1]:
import numpy as np
import pandas as pd
import hashlib
import itertools
import random
import sys
from matplotlib import pyplot as plt
from scipy.sparse import csr_matrix, csc_matrix, coo_matrix, lil_matrix

In [2]:
n_users = 5    # Number of users
n_movies = 5   # Number of movies

# Generate a user-movie rating matrix with random ratings between 1 and 5
ratings = np.random.randint(1, 6, size=(n_users, n_movies))

# Create a DataFrame for better visualization
user_ids = [f"User_{i+1}" for i in range(n_users)]
movie_ids = [f"Movie_{j+1}" for j in range(n_movies)]
user_movie = pd.DataFrame(ratings, index=user_ids, columns=movie_ids)
print(user_movie)

        Movie_1  Movie_2  Movie_3  Movie_4  Movie_5
User_1        4        4        4        4        1
User_2        2        5        2        4        1
User_3        3        4        5        5        2
User_4        1        2        4        4        3
User_5        3        1        5        5        2


In [3]:

def candidates(user_movie, h=200, b=20, seed=1):

    # sparse matrix
    movies_num = int(np.max(user_movie[:, 1]))
    users_num = int(np.max(user_movie[:, 0]))

    sparse_matrix = coo_matrix((np.ones(len(user_movie)), (user_movie[:, 1] - 1, user_movie[:, 0] - 1)))
    sparse_matrix = sparse_matrix.tocsc()

    # signature matrix
    signature_matrix = np.full((h, users_num), np.inf)
    np.random.seed(seed)
    hash_permutations = np.array([np.random.permutation(movies_num) for _ in range(h)])
    rows, cols = sparse_matrix.nonzero()
    for i in range(h):
        permuted_rows = hash_permutations[i, rows]
        np.minimum.at(signature_matrix[i], cols, permuted_rows + 1)

    # print(signature_matrix)
    r = h // b

    # hash into bucket
    bucket_dict_list = []
    for b_ in range(b):
      bucket_dict = dict()
      for u in range(users_num):
        start = r*b_
        end = r*(b_+1)
        data = str(tuple(signature_matrix[start:end, u])).encode('utf-8')
        # bucket_id = hash(data)
        bucket_id = hashlib.sha256(data).hexdigest()
        if bucket_id not in bucket_dict.keys():
          bucket_dict[bucket_id] = [u+1]
        else:
          bucket_dict[bucket_id].append(u+1)
      bucket_dict_list.append(bucket_dict)


    # candidate pairs
    candidate_pairs = set()
    for bucket_dict in bucket_dict_list:
      for bucket_id, bucket in bucket_dict.items():
        if len(bucket) > 1 and len(bucket) < 20:
          candidate_pairs.update(itertools.combinations(bucket, 2))

    candidate_pairs = list(candidate_pairs)
    return candidate_pairs, signature_matrix, sparse_matrix


def calculate_similarity(candidate_pairs):
    pair_counts = []
    user_rated_movies = {user: user_movie[user_movie[:, 0] == user][:,1] for user in np.unique(user_movie[:, 0])}
    for candidates in candidate_pairs:
      pair_count = 0
      for candidate in candidates:
          C1 = user_rated_movies[candidate[0]]
          C2 = user_rated_movies[candidate[1]]
          jsim = len(np.intersect1d(C1, C2)) / len(np.union1d(C1, C2))
          if jsim > 0.5:
              pair_count += 1
      pair_counts.append(pair_count)
    return pair_counts



def calculate_jaccard_similarity(sparse_matrix, candidate_pairs):
    pair_counts = []
    for i, candidates in enumerate(candidate_pairs):
        pair_count = 0
        candidates = np.array(candidates)
        user1_indices = candidates[:, 0] - 1
        user2_indices = candidates[:, 1] - 1

        for user1, user2 in zip(user1_indices, user2_indices):
            C1 = sparse_matrix[:, user1]
            C2 = sparse_matrix[:, user2]
            intersection = C1.minimum(C2).sum()
            union = C1.maximum(C2).sum()
            jsim = intersection / union if union > 0 else 0


            if jsim > 0.5:
                pair_count += 1
        pair_counts.append(pair_count)
    return pair_counts


def calculate_signature_similarity(signature_matrices, candidate_pairs):
    pair_counts = []
    for signature_matrix, candidates in zip(signature_matrices, candidate_pairs):
        pair_count = 0

        candidates = np.array(candidates)
        user1_indices = candidates[:, 0] - 1
        user2_indices = candidates[:, 1] - 1

        C1 = signature_matrix[:, user1_indices]
        C2 = signature_matrix[:, user2_indices]

        agree_matrix = C1 == C2
        agree_counts = np.sum(agree_matrix, axis=0)
        total_rows = signature_matrix.shape[0]

        jsim_values = agree_counts / total_rows

        for jsim in jsim_values:
            if jsim > 0.5:
                pair_count += 1
        pair_counts.append(pair_count)
    return pair_counts


'''def calculate_signature_similarity(signature_matrices, candidate_pairs):
    pair_counts = []
    for signature_matrix, candidates in zip(signature_matrices, candidate_pairs):
        pair_count = 0
        for candidate in candidates:
            user1, user2 = candidate
            C1 = signature_matrix[:, user1 - 1]
            C2 = signature_matrix[:, user2 - 1]
            agree_rows = np.sum(C1 == C2)
            total_rows = signature_matrix.shape[0]
            jsim = agree_rows/total_rows
            if jsim > 0.5:

                pair_count += 1
        pair_counts.append(pair_count)
    return pair_counts'''




if __name__ == "__main__":
    # candidate b
    bs = [20] #, 30, 40, 50
    hs = [100] # 80,
    num_candidates = []
    candidate_pairs = []
    signature_matrices = []
    labels = []

    # random seed
    # randomseed = int(sys.argv[1])
    for h in hs:
      for b in bs:
          candidate_pair, signature_matrix, sparse_matrix = candidates(user_movie, h=h, b=b)
          print(len(candidate_pair))
          labels.append(f"h={h}, b={b}")
          num_candidates.append(len(candidate_pair))
          candidate_pairs.append(candidate_pair)
          signature_matrices.append(signature_matrix)


    '''fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(labels, num_candidates, align='center')
    plt.xlabel('(h, b)')
    plt.ylabel('Number of Candidates')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
'''


    '''pair_count = calculate_similarity(candidate_pairs)
    print(pair_count)'''

    pair_count = calculate_jaccard_similarity(sparse_matrix, candidate_pairs)
    print(pair_count)

    '''pair_count = calculate_signature_similarity(signature_matrices, candidate_pairs)
    print(pair_count)'''



InvalidIndexError: (slice(None, None, None), 1)